In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Exnum/Copy of combined_article_comment_df_10K.csv')
df.head()

,id,title,number,offset,length,magnitude,new_title,source
0,555024,Metro Denver shelters collaborate in no-fee pe...,10.0,63,2,2,Metro Denver shelters collaborate in no-fee pe...,article
1,236417,Learn to Play Ice Skating Classes for Kids 3+,3.0,43,1,1,Learn to Play Ice Skating Classes for Kids [N...,article
2,91618,Tech 101: capturing EVPs with a digital recorder,101.0,5,3,3,Tech [NUM] : capturing EVPs with a digital re...,article
3,53904,Rat City Roller Girl bout 3 this Saturday,3.0,26,1,1,Rat City Roller Girl bout [NUM] this Saturday,article
4,350187,Age Discrimination of People in Their 40s,40.0,38,2,2,Age Discrimination of People in Their [NUM] s,article


In [ ]:
df.magnitude.value_counts()

1    3740
2    2790
4    1250
0    1219
3     940
6      32
5      29
Name: magnitude, dtype: int64

In [ ]:
import pickle

In [ ]:
embeddings=pickle.load(open('/content/drive/MyDrive/Exnum/Copy of embedding-all.pickle',"rb"))

In [ ]:
x=pd.DataFrame(embeddings)
x.shape

(10000, 768)

In [ ]:
x.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.377768,-0.608791,-0.477885,-1.021846,-0.344364,0.544733,0.005602,-0.166941,0.138816,0.466202,...,0.695108,-0.267429,-0.350608,0.891882,0.399979,-0.214678,-0.604156,0.170782,0.015243,1.100375
1,-0.374574,0.533578,0.356899,-0.163032,0.093037,-0.472463,0.099465,1.112708,0.137164,-1.125372,...,1.526861,-0.558944,-0.416206,1.055524,-0.219149,-0.308680,-0.493346,-1.187164,-0.741130,0.325597
2,0.772553,0.175026,-0.015306,-0.694739,0.142113,-0.119007,-0.079949,1.175047,-0.625018,-1.075939,...,1.610133,0.063908,-0.383219,0.399656,0.776549,-0.549381,-0.374127,-0.321196,-0.061886,0.971979
3,0.187604,-0.433557,-0.273567,-0.575823,-0.909251,-0.294007,-0.890702,1.349211,-0.205354,-0.671792,...,0.879788,-0.719193,-0.804741,0.515857,0.019762,-1.000707,-0.638418,-0.727788,0.259793,0.230624
4,-0.355479,-0.456681,0.218642,-0.636924,-0.383086,-0.246211,1.089402,0.636811,-0.518018,-0.887329,...,0.934151,-0.992773,0.140953,1.168294,0.686638,-1.435713,-0.149862,-0.205391,-0.696075,0.871644


In [ ]:
y=df['magnitude']
y.shape

(10000,)

In [ ]:
y.head()

0    2
1    1
2    3
3    1
4    2
Name: magnitude, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import accuracy_score , f1_score

In [ ]:
def get_score(model,x_train,y_train,x_test,y_test):
  model.fit(x_train,y_train)
  y_train_pred=model.predict(x_train)
  train_accu=accuracy_score(y_train,y_train_pred)
  train_f1_micro=f1_score(y_train,y_train_pred,average='micro')
  train_f1_macro=f1_score(y_train,y_train_pred,average='macro')
  y_test_pred=model.predict(x_test)
  test_accu=accuracy_score(y_test,y_test_pred)
  test_f1_micro=f1_score(y_test,y_test_pred,average='micro')
  test_f1_macro=f1_score(y_test,y_test_pred,average='macro')
  return train_accu,train_f1_micro,train_f1_macro,test_accu,test_f1_micro,test_f1_macro

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
fold=StratifiedKFold(n_splits=5)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random forest

In [ ]:
train_ac=[]
train_mif1=[]
train_maf1=[]
test_ac=[]
test_mif1=[]
test_maf1=[]
for train_index,test_index in fold.split(x,y):
  x_train,y_train,x_test,y_test=x.iloc[train_index],y.iloc[train_index],x.iloc[test_index],y.iloc[test_index]
  print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
  model=RandomForestClassifier(max_depth=12, max_leaf_nodes=295, min_samples_split=26,n_jobs=-1)
  tac,tm1,tm2,teac,tem1,tem2=get_score(model,x_train,y_train,x_test,y_test)
  train_ac.append(tac)
  train_mif1.append(tm1)
  train_maf1.append(tm2)
  test_ac.append(teac)
  test_mif1.append(tem1)
  test_maf1.append(tem2)


(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)


In [ ]:
print(train_ac,'\n',test_ac,'\n',train_mif1,'\n',test_mif1,'\n',train_maf1,'\n',test_maf1)

[0.8415, 0.83175, 0.822, 0.807375, 0.818875] 
 [0.5445, 0.5365, 0.5455, 0.495, 0.498] 
 [0.8415, 0.8317499999999999, 0.822, 0.807375, 0.818875] 
 [0.5445, 0.5365, 0.5455, 0.495, 0.498] 
 [0.6595227612676153, 0.6508818851986419, 0.6942951742246299, 0.6155595355459028, 0.6264003536893828] 
 [0.3326422482791667, 0.41021288021182717, 0.46183012035142096, 0.36884508712667735, 0.37730463294146876]


In [ ]:
train_accu=sum(train_ac)/len(train_ac)
train_f1_mi=sum(train_mif1)/len(train_mif1)
train_f1_ma=sum(train_maf1)/len(train_maf1)
test_accu=sum(test_ac)/len(test_ac)
test_f1_mi=sum(test_mif1)/len(test_mif1)
test_f1_ma=sum(test_maf1)/len(test_maf1)
print("training accu :",train_accu)
print("training micro f1 :",train_f1_mi)
print("training macro f1 :",train_f1_ma)
print("test accu :",test_accu)
print("test micro f1 :",test_f1_mi)
print("test macro f1 :",test_f1_ma)

training accu : 0.8243
training micro f1 : 0.8243
training macro f1 : 0.6493319419852346
test accu : 0.5239
test micro f1 : 0.5239
test macro f1 : 0.3901669937821122


logistic regression

In [ ]:
train_ac=[]
train_mif1=[]
train_maf1=[]
test_ac=[]
test_mif1=[]
test_maf1=[]
from sklearn.linear_model import LogisticRegression
for train_index,test_index in fold.split(x,y):
  x_train,y_train,x_test,y_test=x.iloc[train_index],y.iloc[train_index],x.iloc[test_index],y.iloc[test_index]
  print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
  model=LogisticRegression(C=0.05, max_iter=10000, solver='liblinear')
  tac,tm1,tm2,teac,tem1,tem2=get_score(model,x_train,y_train,x_test,y_test)
  train_ac.append(tac)
  train_mif1.append(tm1)
  train_maf1.append(tm2)
  test_ac.append(teac)
  test_mif1.append(tem1)
  test_maf1.append(tem2)

(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)


In [ ]:
print(train_ac,'\n',test_ac,'\n',train_mif1,'\n',test_mif1,'\n',train_maf1,'\n',test_maf1)

[0.674875, 0.673875, 0.6685, 0.67375, 0.68125] 
 [0.547, 0.568, 0.5955, 0.551, 0.524] 
 [0.674875, 0.673875, 0.6685, 0.67375, 0.68125] 
 [0.547, 0.568, 0.5955, 0.551, 0.524] 
 [0.6497000049688939, 0.6642448771588958, 0.6439637176226884, 0.6223226317463421, 0.6392043295730685] 
 [0.3730838723048179, 0.4684586167600954, 0.5995485026197126, 0.49949363690427984, 0.46370620639052146]


In [ ]:
train_accu=sum(train_ac)/len(train_ac)
train_f1_mi=sum(train_mif1)/len(train_mif1)
train_f1_ma=sum(train_maf1)/len(train_maf1)
test_accu=sum(test_ac)/len(test_ac)
test_f1_mi=sum(test_mif1)/len(test_mif1)
test_f1_ma=sum(test_maf1)/len(test_maf1)
print("training accu :",train_accu)
print("training micro f1 :",train_f1_mi)
print("training macro f1 :",train_f1_ma)
print("test accu :",test_accu)
print("test micro f1 :",test_f1_mi)
print("test macro f1 :",test_f1_ma)

training accu : 0.67445
training micro f1 : 0.67445
training macro f1 : 0.6438871122139778
test accu : 0.5571
test micro f1 : 0.5571
test macro f1 : 0.4808581669958854


XGBoost

In [ ]:
from xgboost import XGBClassifier
train_ac=[]
train_mif1=[]
train_maf1=[]
test_ac=[]
test_mif1=[]
test_maf1=[]
for train_index,test_index in fold.split(x,y):
  x_train,y_train,x_test,y_test=x.iloc[train_index],y.iloc[train_index],x.iloc[test_index],y.iloc[test_index]
  print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
  model=XGBClassifier()
  tac,tm1,tm2,teac,tem1,tem2=get_score(model,x_train,y_train,x_test,y_test)
  train_ac.append(tac)
  train_mif1.append(tm1)
  train_maf1.append(tm2)
  test_ac.append(teac)
  test_mif1.append(tem1)
  test_maf1.append(tem2)

(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)


In [ ]:
print(train_ac,'\n',test_ac,'\n',train_mif1,'\n',test_mif1,'\n',train_maf1,'\n',test_maf1)

[0.999875, 0.999875, 0.999875, 0.999875, 1.0] 
 [0.5735, 0.5575, 0.592, 0.558, 0.538] 
 [0.999875, 0.999875, 0.999875, 0.999875, 1.0] 
 [0.5735, 0.5575, 0.592, 0.558, 0.538] 
 [0.9998730690917389, 0.9998730690917389, 0.9998730690917389, 0.9998729571199413, 1.0] 
 [0.3764160579449961, 0.4516700810198938, 0.5515189174712699, 0.5398253559681425, 0.4823410159086821]


In [ ]:
train_accu=sum(train_ac)/len(train_ac)
train_f1_mi=sum(train_mif1)/len(train_mif1)
train_f1_ma=sum(train_maf1)/len(train_maf1)
test_accu=sum(test_ac)/len(test_ac)
test_f1_mi=sum(test_mif1)/len(test_mif1)
test_f1_ma=sum(test_maf1)/len(test_maf1)
print("training accu :",train_accu)
print("training micro f1 :",train_f1_mi)
print("training macro f1 :",train_f1_ma)
print("test accu :",test_accu)
print("test micro f1 :",test_f1_mi)
print("test macro f1 :",test_f1_ma)

training accu : 0.9998999999999999
training micro f1 : 0.9998999999999999
training macro f1 : 0.9998984328790316
test accu : 0.5638
test micro f1 : 0.5638
test macro f1 : 0.48035428566259697


Light GBM

In [ ]:
from lightgbm import LGBMClassifier
train_ac=[]
train_mif1=[]
train_maf1=[]
test_ac=[]
test_mif1=[]
test_maf1=[]
for train_index,test_index in fold.split(x,y):
  x_train,y_train,x_test,y_test=x.iloc[train_index],y.iloc[train_index],x.iloc[test_index],y.iloc[test_index]
  print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
  model=LGBMClassifier(learning_rate=0.01, max_depth=5, num_leaves=30)
  tac,tm1,tm2,teac,tem1,tem2=get_score(model,x_train,y_train,x_test,y_test)
  train_ac.append(tac)
  train_mif1.append(tm1)
  train_maf1.append(tm2)
  test_ac.append(teac)
  test_mif1.append(tem1)
  test_maf1.append(tem2)

(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)
(8000, 768) (8000,) (2000, 768) (2000,)


In [ ]:
print(train_ac,'\n',test_ac,'\n',train_mif1,'\n',test_mif1,'\n',train_maf1,'\n',test_maf1)

[0.68825, 0.676, 0.670875, 0.659875, 0.67025] 
 [0.5125, 0.5265, 0.533, 0.4845, 0.4885] 
 [0.68825, 0.676, 0.670875, 0.659875, 0.67025] 
 [0.5125, 0.5265, 0.533, 0.4845, 0.4885] 
 [0.7614891807668381, 0.7512456374937576, 0.7331322226600542, 0.7334587783519851, 0.7350536752194491] 
 [0.31129545636986317, 0.3302667482607214, 0.4533143539106486, 0.40020776884778114, 0.4213566861383712]


In [ ]:
train_accu=sum(train_ac)/len(train_ac)
train_f1_mi=sum(train_mif1)/len(train_mif1)
train_f1_ma=sum(train_maf1)/len(train_maf1)
test_accu=sum(test_ac)/len(test_ac)
test_f1_mi=sum(test_mif1)/len(test_mif1)
test_f1_ma=sum(test_maf1)/len(test_maf1)
print("training accu :",train_accu)
print("training micro f1 :",train_f1_mi)
print("training macro f1 :",train_f1_ma)
print("test accu :",test_accu)
print("test micro f1 :",test_f1_mi)
print("test macro f1 :",test_f1_ma)

training accu : 0.6730500000000001
training micro f1 : 0.6730500000000001
training macro f1 : 0.7428758988984169
test accu : 0.5090000000000001
test micro f1 : 0.5090000000000001
test macro f1 : 0.38328820270547714
